## Co-axial drone dynamics

<img src="Drone1.png" width="300">

In this exercise, I populate the ```CoaxialCopter``` class with three methods. These functions will calculate vertical acceleration $\ddot{z}$, angular acceleration $\ddot{\psi}$ along the $z$ axis and the angular velocities $\omega_1$ and $\omega_2$ of the propellers required to achieve any desired  $\ddot{z}$ and $\ddot{\psi}$. 

We assume that drone only can travel and rotate along the vertical $z$ axis. 

Remember, the positive $z$ axis points downwards. 

Also notice that the first propeller rotates clockwise, while the second propeller rotates counterclockwise.

**Reminder**: The second propeller rotates counterclockwise thus the angular velocity needs to be positive, while angular velocity of the first propeller needs to be negative as it rotates clockwise.


In [1]:
#%matplotlib inline 
#%config InlineBackend.figure_format = 'retina'

import numpy as np 
import math
#import matplotlib.pyplot as plt
#import matplotlib.pylab as pylab
#import jdc
from ExerciseAnswers import Answers

#pylab.rcParams['figure.figsize'] = 10, 10

#### Helpful Equations

$$F_z = m \ddot{z}$$

$$M_z = I_z \ddot{\psi}$$

$$ \ddot{z} = g- \frac{k_f}{m}\left( \omega_1^2 +\omega_2^2 \right)   \\
   \ddot{\psi} =\frac{k_m}{I_z} (-\omega_1^2 + \omega_2^2 )  $$
   
_Hint_:
From those equations you can derive the following:

$$ \omega_1^2 = \frac{m}{k_f} (g-\ddot{z}) - \omega_2^2 $$

$$ \omega_2^2 = \frac{I_z}{k_m} \ddot{\psi} + \omega_1^2 $$

and replacing the omega_squared variables in the two equations leads to:

$$ \omega_1^2 =\frac{1}{2}\frac{m}{k_f} (g-\ddot{z}) - \frac{1}{2}\frac{I_z}{k_m} \ddot{\psi} $$

$$ \omega_2^2 =\frac{1}{2}\frac{m}{k_f} (g-\ddot{z}) + \frac{1}{2}\frac{I_z}{k_m} \ddot{\psi} $$

$$ $$
$$ $$
_Note_: In the notebook we will refer to 
$ \ddot{\psi} $ as angular_acceleration and 
$\ddot{z}$ as linear_acceleration

In [2]:
class CoaxialCopter:
    
    def __init__(self, 
                 k_f = 0.1, # value of the thrust coefficient
                 k_m = 0.1, # value of the angular torque coefficient
                 m = 0.5,   # mass of the vehicle 
                 i_z = 0.2, # moment of inertia around the z-axis
                ):
        
        self.k_f = k_f
        self.k_m = k_m
        self.m = m
        self.i_z = i_z
        
        self.omega_1 = 0.0
        self.omega_2 = 0.0
        self.g = 9.81
    
    @property
    def z_dot_dot(self):
        """Calculates current vertical acceleration."""
        
        # Calculate lift generated by the propellers (positive downwards)
        lift_1 = self.k_f * self.omega_1 ** 2
        lift_2 = self.k_f * self.omega_2 ** 2
        lift_net = lift_1 + lift_2
        
        # Calculate the total vertical foce on the drone
        Fz = self.m * self.g - lift_net
        
        # Return the vertical acceleration (positive downward)
        return Fz / self.m
    
    @property
    def psi_dot_dot(self): 
        """Calculates current rotational acceleration."""
        
        # Calculate the torques generated by both propellers         
        Tcw = self.k_m * self.omega_1 ** 2
        Tccw = self.k_m * self.omega_2 ** 2
        
        # Return the angular acceleration 
        return (Tcw - Tccw) / self.i_z
    
    def set_rotors_angular_velocities(self, linear_acc, angular_acc): 
        """
        Sets the turn rates for the rotors so that the drone
        achieves the desired linear_acc and angular_acc.
        """
        
        # The second propeller rotates counterclockwise 
        # thus the angular velocity needs to be positive, 
        # while angular velocity of the first propeller needs to be 
        # negative as it rotates clockwise.
        part_1 = 0.5 * (self.g - linear_acc) * self.m / self.k_f
        part_2 = 0.5 * self.i_z * angular_acc / self.k_m 
        
        self.omega_1 = - (part_1 - part_2) ** 0.5
        self.omega_2 = + (part_1 + part_2) ** 0.5

        return self.omega_1, self.omega_2

In [3]:
# TEST CODE 1

bi = CoaxialCopter()
stable_omega_1,stable_omega_2 = bi.set_rotors_angular_velocities(0.0, 0.0)

print('Drone achieves stable hover with angular velocity of %5.2f' % stable_omega_1, 
      'for the first propeller and %5.2f' % stable_omega_2, 
      'for the second propeller.')

Answers.angular_velocities(bi.m, bi.g, bi.i_z, bi.k_f, bi.k_m, 0.0, 0.0, stable_omega_1, stable_omega_2)

Drone achieves stable hover with angular velocity of -4.95 for the first propeller and  4.95 for the second propeller.


'You calculated correct values'

In [4]:
# TEST CODE 2 - Checking the linear acceleration value

bi.omega_1 = stable_omega_1 * math.sqrt(1.1)
bi.omega_2 = stable_omega_2 * math.sqrt(1.1)

vertical_acceleration = bi.z_dot_dot
print('Increase by %5.2f' % math.sqrt(1.1),
      'of the propeller angular velocity will result in',
      '%5.2f' % vertical_acceleration, 
      'm/(s*s) vertical acceleration.' )

Answers.linear_acceleration(bi.m, bi.g, bi.k_f, bi.omega_1, bi.omega_2, vertical_acceleration)

Increase by  1.05 of the propeller angular velocity will result in -0.98 m/(s*s) vertical acceleration.


'You calculated the acceleration value correctly'

In [5]:
# TEST CODE 3 - checking the angular acceleration
bi.omega_1 = stable_omega_1 * math.sqrt(1.1)
bi.omega_2 = stable_omega_2 * math.sqrt(0.9)

ang_acceleration = bi.psi_dot_dot
print('Increase in %5.2f'%math.sqrt(1.1),' of the angular velocity for the first propellr and',
      ' decrease of the angular velocity of the second propellr by %f.2f'%math.sqrt(0.9),' will result in',
      '%5.2f'%ang_acceleration, 'rad/(s*s) angular acceleration.' )

Answers.angular_acceleration(bi.i_z, bi.k_m, bi.omega_1, bi.omega_2, ang_acceleration)

Increase in  1.05  of the angular velocity for the first propellr and  decrease of the angular velocity of the second propellr by 0.948683.2f  will result in  2.45 rad/(s*s) angular acceleration.


'You calculated the angular acceleration correctly'